In [87]:
using Evolutionary
using Flux
using Flux: onehot, onecold, logitcrossentropy #, throttle, @epochs
using MLDatasets
using Random
using StableRNGs

We'll be using Iris dataset for this exmaple

In [88]:
features = Iris.features();
slabels = Iris.labels();
classes = unique(slabels)  # unique classes in the dataset
nclasses = length(classes) # number of classes
d, n = size(features)          # dimension and size if the dataset

(4, 150)

Convert feature and labels in appropriate for training format

In [89]:
data = [ (x, onehot(l, classes)) for (x, l) in zip(eachcol(features), slabels)]

150-element Vector{Tuple{SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, Flux.OneHotArray{UInt32, 3, 0, 1, UInt32}}}:
 ([5.1, 3.5, 1.4, 0.2], [1, 0, 0])
 ([4.9, 3.0, 1.4, 0.2], [1, 0, 0])
 ([4.7, 3.2, 1.3, 0.2], [1, 0, 0])
 ([4.6, 3.1, 1.5, 0.2], [1, 0, 0])
 ([5.0, 3.6, 1.4, 0.2], [1, 0, 0])
 ([5.4, 3.9, 1.7, 0.4], [1, 0, 0])
 ([4.6, 3.4, 1.4, 0.3], [1, 0, 0])
 ([5.0, 3.4, 1.5, 0.2], [1, 0, 0])
 ([4.4, 2.9, 1.4, 0.2], [1, 0, 0])
 ([4.9, 3.1, 1.5, 0.1], [1, 0, 0])
 ([5.4, 3.7, 1.5, 0.2], [1, 0, 0])
 ([4.8, 3.4, 1.6, 0.2], [1, 0, 0])
 ([4.8, 3.0, 1.4, 0.1], [1, 0, 0])
 ⋮
 ([6.0, 3.0, 4.8, 1.8], [0, 0, 1])
 ([6.9, 3.1, 5.4, 2.1], [0, 0, 1])
 ([6.7, 3.1, 5.6, 2.4], [0, 0, 1])
 ([6.9, 3.1, 5.1, 2.3], [0, 0, 1])
 ([5.8, 2.7, 5.1, 1.9], [0, 0, 1])
 ([6.8, 3.2, 5.9, 2.3], [0, 0, 1])
 ([6.7, 3.3, 5.7, 2.5], [0, 0, 1])
 ([6.7, 3.0, 5.2, 2.3], [0, 0, 1])
 ([6.3, 2.5, 5.0, 1.9], [0, 0, 1])
 ([6.5, 3.0, 5.2, 2.0], [0, 0, 1])
 ([6.2, 3.4, 5.4, 2.3], [0, 0, 1

Define some auxiliary functions: model accuracy and its loss

In [90]:
accuracy(model,x,y) = sum(onecold(model(x)) .== onecold(y))/size(x,2)
accuracy(xy, model) = sum( onecold(model(x)) .== onecold(y) for (x,y) in xy) /length(xy)

loss(model) = (x,y)->logitcrossentropy(model(x), y)
loss(model,x,y) = loss(model)(x, y)
loss(xy, model) = loss(model)(hcat(map(first,xy)...), hcat(map(last,xy)...))

loss (generic function with 3 methods)

## Backpropagation

We use a multi-layer perceptron (MLP) model for our classification problem

In [91]:
model = Chain(Dense(d, 15, relu), Dense(15, nclasses))

Chain(
  Dense(4, 15, relu),                   # 75 parameters
  Dense(15, 3),                         # 48 parameters
)                   # Total: 4 arrays, 123 parameters, 748 bytes.

Train the model using the backpropagation method

In [92]:
opt = ADAM(1e-4)
evalcb = Flux.throttle(() -> @show(loss(data, model), accuracy(data, model)), 5)
for i in 1:500
    Flux.train!(loss(model), params(model), data, opt, cb = evalcb)
end

loss(data, model) = 1.1866951880682157
accuracy(data, model) = 0.28


In [93]:
@info "MLP" loss=loss(data, model) accuracy = accuracy(data, model)

┌ Info: MLP
│   loss = 0.08723255779678181
│   accuracy = 0.98
└ @ Main In[93]:1


## Genetic Algorithm

First, we specify a fitness function. We have already defined a loss function for our backpropagation model, so we are going to reuse it.

- We pass an individual to the fitness function to evaluate a loss of the MLP.
- GA optimization searches an individual to minimize the fitness function. In our case optimization direction is aligned with the backpropagation model loss function as we seek to minimize the MLP loss.

In [94]:
fitness(m) = loss(data, m)

fitness (generic function with 1 method)

In [95]:
fitness(model)

0.08723255779678181

### Genetic Operators

We need to define a crossover operator to combine the information of two parents model to generate new individuals. The objective is to increase genetic variability and provide better options.
- Flattent the MLP networks into parameter vector representations
- Perform a crossover operation on the parameter vectors
- Reconstruct MLPs from the parameter vectors

In [96]:
function uniform_mlp(m1::T, m2::T; rng::Random.AbstractRNG=Random.default_rng()) where {T <: Chain}
    θ1, re1 = Flux.destructure(m1);
    θ2, re2 = Flux.destructure(m2);
    c1, c2 = UX(θ1,θ2; rng=rng)
    return re1(c1), re2(c2)
end

uniform_mlp (generic function with 1 method)

In [97]:
uniform_mlp(model, model; rng=StableRNG(42))

(Chain(Dense(4, 15, relu), Dense(15, 3)), Chain(Dense(4, 15, relu), Dense(15, 3)))

Similarly, we rewrite a `gaussian` mutatation operator for MLP.

In [98]:
function gaussian_mlp(σ::Real = 1.0)
    vop = gaussian(σ)
    function mutation(recombinant::T; rng::Random.AbstractRNG=Random.default_rng()) where {T <: Chain}  
        θ, re = Flux.destructure(recombinant)
        return re(convert(Vector{Float32}, vop(θ; rng=rng)))
    end
    return mutation
end

gaussian_mlp (generic function with 2 methods)

In [99]:
gaussian_mlp(0.5)(model; rng=StableRNG(42))

Chain(
  Dense(4, 15, relu),                   # 75 parameters
  Dense(15, 3),                         # 48 parameters
)                   # Total: 4 arrays, 123 parameters, 748 bytes.

### Population

We also require a function for generating a random population of the individuls required for evolutionary optimizations. Our polulation consists of MLP objects, `Flux.Chain` type.

We need to override `Evolutionary.initial_population` which will allows us to create population of the random MPL objects.

In [100]:
import Evolutionary.initial_population
function initial_population(method::M, individual::Chain;
                            rng::Random.AbstractRNG=Random.default_rng(),
                            kwargs...) where {M<:Evolutionary.AbstractOptimizer}
    θ, re = Flux.destructure(individual);
    [re(randn(rng, length(θ))) for i in 1:Evolutionary.population_size(method)]
end

initial_population (generic function with 8 methods)

### Auxiliary functions

The `Evolutionary` optimization algorithms are designed work with a individual population represented as a collection of numerical vectors.
- The optimization objective is kept as a [`EvolutionaryObjective`](https://wildart.github.io/Evolutionary.jl/dev/dev/#Evolutionary.EvolutionaryObjective) object works with any numerical vector type. This object allows to keep a minimizer value, an objective function and its value for minimizer.

In [101]:
EvolutionaryObjective(sum, rand(5))

EvolutionaryObjective{typeof(sum), Float64, Vector{Float64}, Val{:serial}}(sum, 0.0, [NaN, NaN, NaN, NaN, NaN], 0)

However, this type doesn't have a constructor for working with an MLP object, that is of `Flux.Chain` type.

In [102]:
EvolutionaryObjective(fitness, model)

EvolutionaryObjective{typeof(fitness), Float64, Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}}, Val{:serial}}(fitness, 0.08723255779678181, Chain(Dense(4, 15, relu), Dense(15, 3)), 0)

Let's define this missing constructor.

In [103]:
import Evolutionary.EvolutionaryObjective
function EvolutionaryObjective(f, x::Chain; eval::Symbol = :serial)
    fval = f(x)
    EvolutionaryObjective{typeof(f),typeof(fval),typeof(x),Val{eval}}(f, fval, deepcopy(x), 0)
end

EvolutionaryObjective

In [104]:
EvolutionaryObjective(fitness, model)

EvolutionaryObjective{typeof(fitness), Float64, Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}}, Val{:serial}}(fitness, 0.08723255779678181, Chain(Dense(4, 15, relu), Dense(15, 3)), 0)

Additionaly, we are required to make a copy of individuals, but `Flux` doesn't provide a `copy` functions for `Chain` objects, only `deepcopy`. We are going to define some missing functions.
- Test if your individual object successully makes a copy using `copy` functions

In [105]:
import Base: copy

copy(ch::Chain) = deepcopy(ch)

copy (generic function with 168 methods)

### Optimization

Now, we define the parameters of our evolutionary optimization.

In [106]:
opts = Evolutionary.Options(iterations=100, successive_f_tol=25, rng=StableRNG(42))

                  abstol = Inf
                  reltol = Inf
        successive_f_tol = 25
              iterations = 100
             store_trace = false
              show_trace = false
              show_every = 1
                callback = nothing
              time_limit = NaN
         parallelization = serial
                     rng = StableRNGs.LehmerRNG(state=0x00000000000000000000000000000055)


In [107]:
algo = GA(
        selection = rouletteinv,
        mutation =  gaussian_mlp(),
        crossover = uniform_mlp,
        mutationRate = 0.2,
        crossoverRate = 0.9,
        populationSize = 150,
        ε = 0.03
    )

GA[P=150,x=0.9,μ=0.2,ɛ=0.03]

In [108]:
res = Evolutionary.optimize(fitness, model, algo, opts)


 * Status: success

 * Candidate solution
    Minimizer:  Chain(Dense(4, 15, relu), Dense(15, 3))
    Minimum:    0.10592845003287427
    Iterations: 84

 * Found with
    Algorithm: GA[P=150,x=0.9,μ=0.2,ɛ=0.03]

 * Convergence measures
    |f(x) - f(x')| = 0.0 ≤ 1.0e-12

 * Work counters
    Seconds run:   1.2072 (vs limit Inf)
    Iterations:    84
    f(x) calls:    12750


In [109]:
evomodel= Evolutionary.minimizer(res)
@info "MLP" loss=loss(data, evomodel) accuracy = accuracy(data, evomodel)

┌ Info: MLP
│   loss = 0.10592845003287427
│   accuracy = 0.98
└ @ Main In[109]:2
